<a href="https://colab.research.google.com/github/jylee2930/AI_-Introduction-2025-/blob/main/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EC%88%98%ED%95%99_%ED%96%89%EB%A0%AC%EC%97%B0%EC%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##1. 행렬 연산


In [ ]:
##2. 행렬의 이미지표현

import numpy as np
import matplotlib.pyplot as plt
##0,255 영상 보이기
heart = np.array([[1,0,0,1,0,0,1],
                  [0,1,1,0,1,1,0],
                  [0,1,1,1,1,1,0],
                  [1,0,1,1,1,0,1],
                  [1,1,0,1,0,1,1],
                  [1,1,1,0,1,1,1]])

plt.xticks([])
plt.yticks([])
plt.imshow(heart) ##RGB값으로 보이는 함수, cmap = 'gray'
plt.show()

In [ ]:
##빨간색 영상으로 바꿔보자
## 빨간색 = [1,0,0]
height, width = heart.shape
red_heart = np.zeros((height, width, 3))  # RGB 3채널
for i in range(height):
        for j in range(width):
            if heart[i, j] == 1:
                # 빨간색 [R=1, G=0, B=0]
                red_heart[i, j] = [1.0, 0.0, 0.0]
            else:
                # 흰색 [R=1, G=1, B=1]
                red_heart[i, j] = [1.0, 1.0, 1.0]
plt.imshow(red_heart)
plt.show()

In [ ]:
# --- 회전 ---
def rotate(a, k=1, clockwise=True):
    k = k % 4
    return np.rot90(a, k=-k if clockwise else k)

# --- 반전(미러/플립) ---
def flip_lr(a):         # 좌우 반전 (Left-Right)
    return np.fliplr(a)

def flip_ud(a):         # 상하 반전 (Up-Down)
    return np.flipud(a)

def diag_flip(a):       # 주대각선 대칭 (전치)
    return a.T

def anti_diag_flip(a):  # 반대각선 대칭
    return np.rot90(a.T, k=2)

# --- 색(값) 반전: 0<->1 ---
def invert01(a):
    # 원소가 0과 1로만 이루어졌다고 가정
    return 1 - a

In [ ]:
heart = np.array([[1,0,0,1,0,0,1],
                  [0,1,1,0,1,1,0],
                  [0,1,1,1,1,1,0],
                  [1,0,1,1,1,0,1],
                  [1,1,0,1,0,1,1],
                  [1,1,1,0,1,1,1]])
# ---------------- 사용 예시 ----------------
# 시계방향 90도 회전
r90 = rotate(heart, k=1, clockwise=True)
# 반시계방향 180도 회전
r180 = rotate(heart, k=2, clockwise=False)
# 시계방향 280도 회전
r90 = rotate(heart, k=3, clockwise=True)
# 반시계방향 180도 회전
r180 = rotate(heart, k=-2, clockwise=False)

lr   = flip_lr(heart)
ud   = flip_ud(heart)
d1   = diag_flip(heart)
d2   = anti_diag_flip(heart)
inv  = invert01(heart)

In [ ]:
plt.imshow(r90)
plt.show()

In [ ]:
plt.imshow(lr)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
##0~255 흑백영상 보이기
A = np.array([[77,114,140,191],
              [39,56,46,119],
              [61,29,20,30],
              [151,97,43,26]])

plt.xticks([])
plt.yticks([])
plt.imshow(A,cmap = 'gray',vmin=0,vmax=255)
plt.show()


# 합성곱 신경망(CNN)

In [ ]:
##1. 이미지 처리와 필터링
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm


# 1. 간단한 컬러 이미지 생성 (8x8 크기)
def create_sample_image():
    """빨강, 초록, 파랑이 섞인 간단한 테스트 이미지 생성"""
    img = np.zeros((8, 8, 3), dtype=np.uint8)

    # 빨간색 영역
    img[0:4, 0:4] = [255, 0, 0]
    # 초록색 영역
    img[0:4, 4:8] = [0, 255, 0]
    # 파란색 영역
    img[4:8, 0:4] = [0, 0, 255]
    # 노란색 영역 (빨강 + 초록)
    img[4:8, 4:8] = [255, 255, 0]
    return img

 # 원본 이미지 생성
original = create_sample_image()
plt.imshow(original)
plt.show()

In [ ]:
# 2. 마스크를 이용한 색상 변환
def apply_color_mask(image, mask):
    """
    마스크를 이용해 색상 변환
    mask: [R계수, G계수, B계수] 형태
    예: [1.0, 0.5, 0.3]은 빨강은 그대로, 초록은 50%, 파랑은 30%로 감소
    """
    result = image.copy().astype(float)
    result[:, :, 0] *= mask[0]  # Red 채널
    result[:, :, 1] *= mask[1]  # Green 채널
    result[:, :, 2] *= mask[2]  # Blue 채널

    # 0-255 범위로 클리핑
    result = np.clip(result, 0, 255).astype(np.uint8)
    return result

# 3. 컨볼루션을 이용한 다운샘플링 (화소 줄이기)
def apply_convolution_downsampling(image, kernel_size=2):
    """
    컨볼루션(평균 필터)을 이용한 다운샘플링
    kernel_size x kernel_size 영역의 평균을 구해 화소를 줄임
    """
    h, w, c = image.shape
    new_h = h // kernel_size
    new_w = w // kernel_size

    # 결과 이미지
    result = np.zeros((new_h, new_w, c), dtype=np.uint8)

    # 각 채널에 대해 평균 필터 적용
    for i in range(new_h):
        for j in range(new_w):
            # kernel_size x kernel_size 영역의 평균 계산
            region = image[i*kernel_size:(i+1)*kernel_size,
                          j*kernel_size:(j+1)*kernel_size, :]
            result[i, j, :] = np.mean(region, axis=(0, 1))

    return result

In [ ]:
# 원본 이미지 생성
original = create_sample_image()

# 예제 1: 여러 가지 색상 마스크 적용
mask1 = [1.0, 0.5, 0.3]  # 빨강 강조, 초록/파랑 감소
mask2 = [0.3, 1.0, 0.5]  # 초록 강조
mask3 = [0.5, 0.3, 1.0]  # 파랑 강조

masked1 = apply_color_mask(original, mask1)
masked2 = apply_color_mask(original, mask2)
masked3 = apply_color_mask(original, mask3)

# 예제 2: 다운샘플링
downsampled_2x = apply_convolution_downsampling(original, kernel_size=2)

plt.imshow(original)
plt.show()


In [ ]:
plt.imshow(masked1)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
from scipy import signal # scipy.signal 모듈 추가

# 1. 간단한 입력 영상 생성 (8x8)
input_image_pil = Image.open('/content/강아지.JPG')
# 이미지를 흑백으로 변환하고 numpy 배열로 변경
input_image = np.array(input_image_pil.convert('L'))

# 2. 합성곱 필터 정의 (3x3)
# 수직 엣지 검출 필터
vertical_filter = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
])

# 수평 엣지 검출 필터
hydrogen_filter = np.array([
    [-1, -1, -1],
    [ 0,  0,  0],
    [ 1,  1,  1]
])

# 3. 합성곱 연산 수행
conv_vertical = signal.correlate2d(input_image, vertical_filter, mode='valid')
conv_horizontal = signal.correlate2d(input_image, hydrogen_filter, mode='valid')

# 4. 다운샘플링 (Max Pooling 2x2)
def max_pooling(image, pool_size=2):
    h, w = image.shape
    h_out = h // pool_size
    w_out = w // pool_size
    output = np.zeros((h_out, w_out))

    for i in range(h_out):
        for j in range(w_out):
            output[i, j] = np.max(
                image[i*pool_size:(i+1)*pool_size,
                      j*pool_size:(j+1)*pool_size]
            )
    return output

pooled_vertical = max_pooling(conv_vertical)
pooled_horizontal = max_pooling(conv_horizontal)

# 5. 시각화
fig, axes = plt.subplots(3, 3, figsize=(12, 12))

# 첫 번째 행: 입력과 필터
axes[0, 0].imshow(input_image, cmap='gray', vmin=0, vmax=255) # vmax를 255로 수정
axes[0, 0].set_title('Image (8x8)', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')
# 이미지 크기에 따라 텍스트 표시 범위 조절
for i in range(input_image.shape[0]):
    for j in range(input_image.shape[1]):
        axes[0, 0].text(j, i, f'{input_image[i,j]:.0f}',
                       ha='center', va='center', color='red', fontsize=8)

axes[0, 1].imshow(vertical_filter, cmap='RdBu', vmin=-1, vmax=1)
axes[0, 1].set_title('Vertical_Edge_filter (3x3)', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')
for i in range(3):
    for j in range(3):
        axes[0, 1].text(j, i, f'{vertical_filter[i,j]:.0f}',
                       ha='center', va='center', color='black', fontsize=10)

axes[0, 2].imshow(horizontal_filter, cmap='RdBu', vmin=-1, vmax=1)
axes[0, 2].set_title('Horizontal_Edge_filter (3x3)', fontsize=12, fontweight='bold')
axes[0, 2].axis('off')
for i in range(3):
    for j in range(3):
        axes[0, 2].text(j, i, f'{horizontal_filter[i,j]:.0f}',
                       ha='center', va='center', color='black', fontsize=10)

# 두 번째 행: 합성곱 결과
axes[1, 0].axis('off')

axes[1, 1].imshow(conv_vertical, cmap='RdBu')
axes[1, 1].set_title(f'Conv_Vertical_Result ({conv_vertical.shape[0]}x{conv_vertical.shape[1]})',
                     fontsize=12, fontweight='bold')
axes[1, 1].axis('off')
for i in range(conv_vertical.shape[0]):
    for j in range(conv_vertical.shape[1]):
        axes[1, 1].text(j, i, f'{conv_vertical[i,j]:.0f}',
                       ha='center', va='center', color='white', fontsize=7)

axes[1, 2].imshow(conv_horizontal, cmap='RdBu')
axes[1, 2].set_title(f'Conv_Horizontal_Result ({conv_horizontal.shape[0]}x{conv_horizontal.shape[1]})',
                     fontsize=12, fontweight='bold')
axes[1, 2].axis('off')
for i in range(conv_horizontal.shape[0]):
    for j in range(conv_horizontal.shape[1]):
        axes[1, 2].text(j, i, f'{conv_horizontal[i,j]:.0f}',
                       ha='center', va='center', color='white', fontsize=7)

# 세 번째 행: 다운샘플링 결과
axes[2, 0].axis('off')

axes[2, 1].imshow(pooled_vertical, cmap='RdBu')
axes[2, 1].set_title(f'Max Pooling Result ({pooled_vertical.shape[0]}x{pooled_vertical.shape[1]})',
                     fontsize=12, fontweight='bold')
axes[2, 1].axis('off')
for i in range(pooled_vertical.shape[0]):
    for j in range(pooled_vertical.shape[1]):
        axes[2, 1].text(j, i, f'{pooled_vertical[i,j]:.0f}',
                       ha='center', va='center', color='white', fontsize=10)

axes[2, 2].imshow(pooled_horizontal, cmap='RdBu')
axes[2, 2].set_title(f'Max Pooling Result ({pooled_horizontal.shape[0]}x{pooled_horizontal.shape[1]})',
                     fontsize=12, fontweight='bold')
axes[2, 2].axis('off')
for i in range(pooled_horizontal.shape[0]):
    for j in range(pooled_horizontal.shape[1]):
        axes[2, 2].text(j, i, f'{pooled_horizontal[i,j]:.0f}',
                       ha='center', va='center', color='white', fontsize=10)

plt.suptitle('CNN_Examples', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()


# 상세 설명 출력
print("\n" + "="*60)
print("📊 CNN 동작 과정 설명")
print("="*60)
print(f"\n1️⃣ 입력 영상: {input_image.shape} (사각형 형태)")
print(f"2️⃣ 합성곱 필터: {vertical_filter.shape}")
print(f"3️⃣ 합성곱 결과: {conv_vertical.shape}")
print(f"4️⃣ Max Pooling 결과: {pooled_vertical.shape}")
print("\n💡 크기 변화: 8x8 → 6x6 (합성곱) → 3x3 (풀링)")
print("="*60)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

# 컨볼루션 연산 함수
def apply_convolution(image, kernel):
    """
    이미지에 필터(커널)를 적용하는 함수

    Parameters:
    - image: 입력 이미지 (2D numpy array)
    - kernel: 필터 (2D numpy array)

    Returns:
    - feature_map: 필터 적용 후 특성 맵
    """
    # 이미지와 커널의 크기
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    # 출력 이미지 크기 계산
    output_height = image_height - kernel_height + 1
    output_width = image_width - kernel_width + 1

    # 특성 맵 초기화
    feature_map = np.zeros((output_height, output_width))

    # 컨볼루션 연산
    for i in range(output_height):
        for j in range(output_width):
            # 이미지의 일부분 추출
            region = image[i:i+kernel_height, j:j+kernel_width]
            # 필터와 곱한 후 합계
            feature_map[i, j] = np.sum(region * kernel)

    return feature_map


# 1. 업로드된 강아지 이미지 불러오기
print("🐕 이미지 필터 적용 데모\n")

# 업로드된 이미지 불러오기
try:
    dog_image = Image.open('/content/강아지.JPG')
    # 그레이스케일로 변환
    dog_image_gray = dog_image.convert('L')
    # numpy 배열로 변환
    test_image = np.array(dog_image_gray)
    print(f"✅ 강아지 이미지 로드 완료! 크기: {test_image.shape}")
except Exception as e:
    print(f"⚠️ 이미지 로드 실패: {e}")
    print("대체 이미지 사용")
    # 대체 이미지 생성
    test_image = np.array([
        [50, 50, 50, 50, 50, 50, 50, 50],
        [50, 100, 100, 100, 100, 100, 100, 50],
        [50, 100, 200, 200, 200, 200, 100, 50],
        [50, 100, 200, 255, 255, 200, 100, 50],
        [50, 100, 200, 255, 255, 200, 100, 50],
        [50, 100, 200, 200, 200, 200, 100, 50],
        [50, 100, 100, 100, 100, 100, 100, 50],
        [50, 50, 50, 50, 50, 50, 50, 50]
    ])

# 2. 필터 정의 (예시의 3x3 필터)
# 엣지 검출 필터 (Edge Detection Filter)
edge_filter = np.array([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
])

print("📊 필터 (Filter/Kernel):")
print(edge_filter)
print()

# 3. 컨볼루션 적용
feature_map = apply_convolution(test_image, edge_filter)

print("✅ 컨볼루션 연산 완료!")
print(f"입력 이미지 크기: {test_image.shape}")
print(f"필터 크기: {edge_filter.shape}")
print(f"특성 맵 크기: {feature_map.shape}")
print()

# 4. 시각화
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

# 원본 이미지
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Input Image\n(Original)', fontsize=14, fontweight='bold')
axes[0].axis('off')
axes[0].text(0.5, -0.1, f'Size: {test_image.shape[0]}x{test_image.shape[1]}',
             ha='center', transform=axes[0].transAxes)

# 필터
axes[1].imshow(edge_filter, cmap='RdBu', vmin=-1, vmax=8)
axes[1].set_title('Filter (Kernel)', fontsize=14, fontweight='bold')
axes[1].axis('off')
# 필터 값 표시
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, str(edge_filter[i, j]),
                    ha='center', va='center', fontsize=16, fontweight='bold')
axes[1].text(0.5, -0.1, f'Size: 3x3',
             ha='center', transform=axes[1].transAxes)

# 특성 맵
im = axes[2].imshow(feature_map, cmap='gray')
axes[2].set_title('Feature Map\n(After Convolution)', fontsize=14, fontweight='bold')
axes[2].axis('off')
axes[2].text(0.5, -0.1, f'Size: {feature_map.shape[0]}x{feature_map.shape[1]}',
             ha='center', transform=axes[2].transAxes)

plt.colorbar(im, ax=axes[2], fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig('/content/convolution_demo.png', dpi=300, bbox_inches='tight')
print("✅ 이미지 저장 완료: convolution_demo.png\n")

# 5. 다양한 필터 예시
print("="*60)
print("📚 다양한 필터 예시")
print("="*60)

# 여러 종류의 필터 정의
filters = {
    'Edge Detection': np.array([[-1, -1, -1],
                                [-1,  8, -1],
                                [-1, -1, -1]]),

    'Blur (Average)': np.array([[1/9, 1/9, 1/9],
                                [1/9, 1/9, 1/9],
                                [1/9, 1/9, 1/9]]),

    'Sharpen': np.array([[ 0, -1,  0],
                        [-1,  5, -1],
                        [ 0, -1,  0]]),

    'Vertical Edge': np.array([[-1, 0, 1],
                               [-1, 0, 1],
                               [-1, 0, 1]]),

    'Horizontal Edge': np.array([[-1, -1, -1],
                                 [ 0,  0,  0],
                                 [ 1,  1,  1]])
}

# 각 필터 적용 결과 시각화
fig, axes = plt.subplots(2, 3, figsize=(10, 5))
axes = axes.flatten()

# 원본 이미지
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0].axis('off')

# 각 필터 적용
for idx, (filter_name, filter_kernel) in enumerate(filters.items(), 1):
    result = apply_convolution(test_image, filter_kernel)
    axes[idx].imshow(result, cmap='gray')
    axes[idx].set_title(filter_name, fontsize=12, fontweight='bold')
    axes[idx].axis('off')

    print(f"\n{filter_name}:")
    print(filter_kernel)

plt.tight_layout()
plt.savefig('', dpi=300, bbox_inches='tight')
print("\n✅ 다양한 필터 결과 저장 완료: various_filters.png")

# 6. 컨볼루션 과정 상세 설명
print("\n" + "="*60)
print("🔍 컨볼루션 연산 과정 상세 설명")
print("="*60)

# 작은 예시로 단계별 설명
small_image = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])

simple_filter = np.array([
    [1, 0],
    [0, 1]
])

print("\n입력 이미지 (4x4):")
print(small_image)
print("\n필터 (2x2):")
print(simple_filter)
print("\n컨볼루션 과정:")

output_h = small_image.shape[0] - simple_filter.shape[0] + 1
output_w = small_image.shape[1] - simple_filter.shape[1] + 1
result = np.zeros((output_h, output_w))

for i in range(output_h):
    for j in range(output_w):
        region = small_image[i:i+2, j:j+2]
        value = np.sum(region * simple_filter)
        result[i, j] = value
        print(f"\n위치 ({i},{j}):")
        print(f"이미지 영역:\n{region}")
        print(f"필터와 곱셈: {region.flatten()} * {simple_filter.flatten()}")
        print(f"합계: {value}")

print("\n최종 특성 맵 (Feature Map):")
print(result.astype(int))

print("\n" + "="*60)
print("🎉 데모 완료!")
print("="*60)

## 다항식 미분을 활용한 인공지능 활용
##1. 1차 미분만을 사용해 함수를 최정화시키자. 경사하강법

In [ ]:
## 파이썬으로 미분할 함수와 미분할 변수를 이용하여 미분계산하기
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
##
x = sym.Symbol('x')
y = 2*x**3 + 3*x**2 + x + 1
first_diff_y = sym.diff(y, x )
second_diff_y = sym.diff(y, x,2)

print(f"일차미분함수 : {first_diff_y}, 이차미분함수: {second_diff_y}")

In [ ]:
# 임계점 찾기 (1차 도함수 = 0인 점)
critical_points = sym.solve(first_diff_y, x)
print(f"임계점 : {critical_points}")


In [ ]:
##특성화맵

